In [2]:
#导入各种需要的库
#导入各种需要的库
import geemap
import ee
Map = geemap.Map()
from IPython.display import Image
import PySimpleGUI as sg
from tqdm.notebook import tqdm
from time import sleep
import csv
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [*] %(processName)s %(message)s"
)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
js_snippet = """
// Define a rectangle geometry around San Francisco.
var roi = ee.Geometry.Rectangle([-122.45, 37.74, -122.4, 37.8]);
Map.addLayer(roi, {'color':'grey'}, 'roi');
// Map.centerObject(roi);

// Import a Landsat 8 TOA image for this region.
var img = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318');
Map.addLayer(img, {'bands':['B4','B3','B2'],'min':0,'max':0.3}, 'img');

var constant = ee.Image(1);
var xVar = img.select('B5');
var yVar = img.select('B6');
var imgRegress = ee.Image.cat(constant, xVar, yVar);

var linearRegression = imgRegress.reduceRegion({
  reducer: ee.Reducer.linearRegression({
    numX: 2,
    numY: 1
  }),
  geometry: roi,
  scale: 30,
});

// Convert the coefficients array to a list.
var coefList = ee.Array(linearRegression.get('coefficients')).toList();

// Extract the y-intercept and slope.
var b0 = ee.List(coefList.get(0)).get(0); // y-intercept
var b1 = ee.List(coefList.get(1)).get(0); // slope

// Extract the residuals.
var residuals = ee.Array(linearRegression.get('residuals')).toList().get(0);

// Inspect the results.
print('OLS estimates', linearRegression);
print('y-intercept:', b0);
print('Slope:', b1);
print('Residuals:', residuals);
"""
geemap.js_snippet_to_py(
    js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [12]:
# Define a rectangle geometry around San Francisco.
roi = ee.Geometry.Rectangle([-122.45, 37.74, -122.4, 37.8])
Map.addLayer(roi, {'color':'grey'}, 'roi')
# Map.centerObject(roi)

# Import a Landsat 8 TOA image for this region.
img = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
Map.addLayer(img, {'bands':['B4','B3','B2'],'min':0,'max':0.3}, 'img')

constant = ee.Image(1)
xVar = img.select('B5')
yVar = img.select('B6')
imgRegress = ee.Image.cat(constant, xVar, yVar)

linearRegression = imgRegress.reduceRegion(**{
  'reducer': ee.Reducer.linearRegression(**{
    'numX': 2,
    'numY': 1
  }),
  'geometry': roi,
  'scale': 30,
})

# Convert the coefficients array to a list.
coefList = ee.Array(linearRegression.get('coefficients')).toList()

# Extract the y-intercept and slope.
b0 = ee.List(coefList.get(0)).get(0); # y-intercept
b1 = ee.List(coefList.get(1)).get(0); # slope

# Extract the residuals.
residuals = ee.Array(linearRegression.get('residuals')).toList().get(0)

# Inspect the results.
print('OLS estimates', linearRegression.getInfo())
print('y-intercept:', b0.getInfo())
print('Slope:', b1.getInfo())
print('Residuals:', residuals.getInfo())



OLS estimates {'coefficients': [[0.037021137878008106], [0.6358535093372875]], 'residuals': [0.025795838155544124]}
y-intercept: 0.037021137878008106
Slope: 0.6358535093372875
Residuals: 0.025795838155544124
